In [8]:
!pip install jupyter-black

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.0 MB/s eta 0:00:00


In [14]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

%load_ext rich
%load_ext jupyter_black

The rich extension is already loaded. To reload it, use:
  %reload_ext rich
The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [2]:
raw_dataset  = load_dataset("glue", "mrpc")
raw_dataset

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [3]:
raw_train  = raw_dataset["train"]
raw_train[0]


{
    'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
    'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
    'label': 1,
    'idx': 0
}

In [4]:
raw_train.features


{
    'sentence1': Value('string'),
    'sentence2': Value('string'),
    'label': ClassLabel(names=['not_equivalent', 'equivalent']),
    'idx': Value('int32')
}

In [5]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tk1 = tokenizer(raw_train[0]["sentence1"])
tk2 = tokenizer(raw_train[0]["sentence2"])

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [6]:
inputs = tokenizer("This is the first sentence.", "This is the second one.")
inputs


{
    'input_ids': [
        101,
        2023,
        2003,
        1996,
        2034,
        6251,
        1012,
        102,
        2023,
        2003,
        1996,
        2117,
        2028,
        1012,
        102
    ],
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}

In [7]:
tokenizer.convert_ids_to_tokens(inputs["input_ids"])


[
    '[CLS]',
    'this',
    'is',
    'the',
    'first',
    'sentence',
    '.',
    '[SEP]',
    'this',
    'is',
    'the',
    'second',
    'one',
    '.',
    '[SEP]'
]

In [11]:
tokenized_dataset = tokenizer(
    raw_dataset["train"]["sentence1"],
    raw_dataset["train"]["sentence2"],
    truncation=True,
    padding=True,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:1                                                                              │
│                                                                                                  │
│ ❱ 1 tokenized_dataset = tokenizer(                                                               │
│   2 │   raw_dataset["train"]["sentence1"], raw_dataset["train"]["sentence2"], truncation=Tru     │
│   3 )                                                                                            │
│   4                                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2911 in __call__ │
│                                                                                                  │
│   2908 │   │   │   # input mode in this case.                                                    │
│   2909 │   │   │   if not self._in_target_context_manager:                                       │
│   2910 │   │   │   │   self._switch_to_input_mode()                                              │
│ ❱ 2911 │   │   │   encodings = self._call_one(text=text, text_pair=text_pair, **all_kwargs)      │
│   2912 │   │   if text_target is not None:                                                       │
│   2913 │   │   │   self._switch_to_target_mode()                                                 │
│   2914 │   │   │   target_encodings = self._call_one(text=text_target, text_pair=text_pair_targ  │
│                                                                                                  │
│ /usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2971 in          │
│ _call_one                                                                                        │
│                                                                                                  │
│   2968 │   │   │   │   return False                                                              │
│   2969 │   │                                                                                     │
│   2970 │   │   if not _is_valid_text_input(text):                                                │
│ ❱ 2971 │   │   │   raise ValueError(                                                             │
│   2972 │   │   │   │   "text input must be of type `str` (single example), `list[str]` (batch o  │
│   2973 │   │   │   │   "or `list[list[str]]` (batch of pretokenized examples)."                  │
│   2974 │   │   │   )                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: text input must be of type `str` (single example), `list[str]` (batch or single pretokenized example) 
or `list[list[str]]` (batch of pretokenized examples).

In [12]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [13]:
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]


DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [15]:
data_collater = DataCollatorWithPadding(tokenizer=tokenizer)